In [ ]:
# load annotations
import json
from pycocotools.coco import COCO
import os

In [ ]:
jsonfiles = ['/root/data/small_pen_data_collection/body_parts_detection_part0.json',
             '/root/data/small_pen_data_collection/body_parts_detection_part1.json']

In [ ]:
full_fish = []
for jf in jsonfiles:
    cocofile = COCO(jf)
    for (image_id, image_data) in cocofile.imgs.items():
        # load annotation (only head and caudal fin)
        annotation_ids = cocofile.getAnnIds(imgIds=[image_id], catIds=[1, 2], iscrowd=None)
        annotations = cocofile.loadAnns(annotation_ids)

        unique_categories = list(set([a['category_id'] for a in annotations]))
        if 1 in unique_categories and 2 in unique_categories:
            # then we have detected head and caudal fin
            full_fish.append(image_data['local_path'])

In [ ]:
print(len(full_fish))

## Check pairs

In [ ]:
timestamps = dict()
for f in full_fish:
    ts = os.path.basename(f).split('.')[0].split('_')[-1]
    side = os.path.basename(f).split('.')[0].split('_')[0]
    if ts not in timestamps:
        timestamps[ts] = dict()
    timestamps[ts][side] = f

In [ ]:
print("number of unique timestamps: {}".format(len(timestamps.keys())))
print("number of left images: {}".format(len([v for (k, v) in timestamps.items() if "left" in v])))
print("number of right images: {}".format(len([v for (k, v) in timestamps.items() if "right" in v])))
print("number of pairs: {}".format(len([v for (k, v) in timestamps.items() if "right" in v and "left" in v])))

In [ ]:
experiments = {}
for (ts, sidedict) in timestamps.items():
    if 'left' in sidedict and 'right' in sidedict:
        exp = sidedict['left'].split('/')[-2]
        if exp not in experiments:
            experiments[exp] = []
        experiments[exp].append((sidedict['left'], sidedict['right']))

In [ ]:
for (k, v) in experiments.items():
    print("Number of good pairs in experiment {}: {}".format(k, len(v)))

# copy files for annotations

In [ ]:
import shutil
import numpy as np

In [ ]:
len(timestamps.keys())

In [ ]:
for (k, v) in timestamps.items():
    if 'left' not in v:
        continue
    if 'right' not in v:
        continue
    coin = np.random.rand()
    if coin > 0.25:
        continue
    # copy left
    src = v['left']
    dst = os.path.join('/root/data/small_pen_data_collection/pairs_for_annotations/', os.path.basename(src))
    shutil.copy(src, dst)
    
    # copy right
    src = v['right']
    dst = os.path.join('/root/data/small_pen_data_collection/pairs_for_annotations/', os.path.basename(src))
    shutil.copy(src, dst)